In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

## Retail Protect 90 Fund

In [ ]:
df = pd.read_csv("data.csv", parse_dates=True, index_col = "Dates",sep=';')
print(f"Nombre d'échantillons : {len(df)}")
df

In [ ]:
sxxr = df.iloc[:,1]
sxxr = pd.DataFrame(sxxr, columns = ['SXXR Index'])
eonia = df.iloc[:,0]
eonia = pd.DataFrame(eonia, columns = ["EONIA Index"])

In [ ]:
# Création de la figure et des sous-graphiques
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

# Tracé des graphiques à gauche
ax1.plot(sxxr, c="r")
ax1.set_title('STOXX EUROPE 600 Index')
ax1.grid(True)

ax2.axhline(0, color='black', linewidth=2, linestyle='-')
ax2.plot(eonia,c="r")
ax2.set_title('EONIA Index')
ax2.grid(True)

# Affichage de la figure
plt.tight_layout()
plt.show()

## Préparation des données

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
def monetarize(df:np.ndarray, nav:float, year:int, horizon:int=0)->pd.DataFrame:
    # Années précédente, à prendre le dernier jour
    tab_prec = df[str(year-1):str(year-1)]
    tab_prec["Loan"] = 100
    tab_prec = pd.DataFrame(tab_prec.iloc[-1]).T
    
    # Années en cours sur un horizon donné
    tab_suiv = df[str(year):str(year+horizon)]
    tab_suiv["Loan"] = 100
    
    tab = pd.concat([tab_prec, tab_suiv])

    rate = [x for x in tab.iloc[:,0]]
    table = np.zeros((len(rate)+1,1))
    table[0] = nav
    compteur = 0
    for elem in rate:
        compteur+=1
        table[compteur] = np.round(table[compteur-1] * (1 + elem/(365*100)),2)
    tab["Loan"] = np.round(table[0:-1],2)
    tab = tab.drop("EONIA Index", axis = 1)
    
    # Year Y
    
    tabl = tab[str(year):str(year+horizon)]
    
    # Year Y-1
    yearn = year-1
    tabn = tab[str(yearn):str(yearn)]
    yearn_last_day = pd.DataFrame(tabn.iloc[-1]).T
    
    # Concatenate    
    tabulate = pd.concat([yearn_last_day,tabl])
    
    tabulate["EONIA Index"] = tabulate
    tabulate = tabulate.drop("Loan", axis = 1)
    
    return tabulate  

def particularyear(df:pd.DataFrame,year):
    tab = df
    
    # Year Y-1
    yearn = year-1
    tabn = tab[str(yearn):str(yearn)]
    yearn_last_day = pd.DataFrame(tabn.iloc[-1]).T

    # Year Y
    tabl = tab[str(year):str(year)]  
    
    # Concatenate    
    tabulate = pd.concat([yearn_last_day,tabl])
    
    return tabulate     
    
    
def normalize(df:pd.DataFrame, year:int, horizon:int=0)->pd.DataFrame:
    """
    This function has only one utility which is to normalize
    the dataframe df (which is the sxxr one) by dividing all the
    dataframe by the first value of sxxr.
    """
    
    S0 = pd.DataFrame(df.iloc[0]).T
    df = round((df/(float(S0.iloc[0,0])))*100,3)
    
    # Year Y-1
    tabn = df[str(year-1):str(year-1)]
    year_last_day = round(pd.DataFrame(tabn.iloc[-1]).T,3)
    
    # Year Y
    tab = df[str(year):str(year+horizon)]
    
    # Concatenate
    tabulate = pd.concat([year_last_day, tab])

    return tabulate

def concat(df1:pd.DataFrame, df2:pd.DataFrame):
    if all(df1.index == df2.index):
        tab = pd.concat([df1,df2],axis =1 )
    else:
        print("Please, be sure both DataFrame have the same index")
    return tab

def VL(df:pd.DataFrame, w:float):
    """
    This function allows to create the NAV dataframe
    The weight "w" should be between 0 and 1
    """
    df1 = pd.DataFrame(df.iloc[:,0]) # eonia
    df2 = pd.DataFrame(df.iloc[:,1]) # sxxr
    n = len(df)
    zer = np.zeros((n,1))
    df["VL"] = zer
    df["VL"] = df["EONIA Index"] * (1-w) + w * df["SXXR Index"]
    val = pd.DataFrame(df["VL"])
    return val

def get_list_protected(df:pd.DataFrame, year:int, plancher:float=0.9, horizon:int=0):
    tabn = df[str(year-1):str(year-1)]
    tabn = pd.DataFrame(tabn.iloc[-1]).T
    tabz = df[str(year):str(year+horizon)]
    tab = pd.concat([tabn, tabz])
    all_plancher = []
    elemlist = []
    for elem in tab.iloc[:,0]:
        elemlist.append(elem)
        if elem == np.max(elemlist):
            planch = plancher * elem
            all_plancher.append(planch)
        else:
            planch = all_plancher[-1]
            all_plancher.append(planch)
            
    #result
    tab["Plancher"] = all_plancher
    tab["Coussin"] = tab["VL"] - tab["Plancher"]
        
    return tab

w = 0.9
multiplier = 1/w

def protection(df:pd.DataFrame, m:multiplier = multiplier):
    tab = df
    # Initialisation des autres colonnes :
    tab["m"] = multiplier
    tab["Risk"] = 0
    tab["No Risk"] = 0
    n = len(tab)
    rang_couss = range(1,n+1)
    tab["Rang Coussin"] = rang_couss

    # Recherche de la première ligne à laquelle le coussin est négative ou nulle
    tabtravail = tab[["Coussin", "Rang Coussin"]]
    liste = []
    date = []
    compteur = 0
    for i in tabtravail.iloc[:,0]:
        compteur += 1
        if i <=0:
            liste.append(compteur)
            dateJ = tabtravail.iloc[compteur-1:compteur].index[0]
            date.append(dateJ)
    date_critique = date[0] # Au cas où l'on souhaite connaitre la date
    coussnegatif = min(liste)-1  # Le rang auquel le premier coussin soit négatif ou nul, -1 car ca commence à 0
    
    if len(date) > 0: # S'il existe une date à laquelle le coussin est négatif ou nul    
        # Mise à jour des colonnes Risk et No Risk

        risk = [x for x in tab["SXXR Index"]]
        norisk = [x for x in tab["EONIA Index"]]

        tab["Risk"] = risk
        tab["No Risk"] = norisk

        allocation = tab[["Risk", "No Risk", "EONIA"]]

        allocation = allocation.iloc[coussnegatif:,]
        allocation["No Risk"] = (1/m)*allocation["Risk"] + (1-1/m)*allocation["No Risk"]
        allocation["Risk"] = 0
    
        # Investir totalement sur l'actif non risqué
        rate = [x for x in allocation["EONIA"]]
        actnorisk = [x for x in allocation["No Risk"]]

        for i in range(len(allocation)):
            if i == 0:
                actnorisk[i] = actnorisk[i]
            else:
                actnorisk[i] = actnorisk[i-1] * (1 + rate[i-1]/(365*100))

        allocation["Actualized no risk"] = actnorisk 
        
        # concat
        
        vl = tab["VL"]
        vl = vl.iloc[:coussnegatif,]
        before = [round(x,3) for x in vl]
        
        after = [round(x,3) for x in allocation["Actualized no risk"]]
        
        all_vl = before + after
        
    #tab = tab.drop(["VL", "SXXR Index", "EONIA Index", "EONIA", "m", "Risk", "No Risk", "Rang Coussin"], axis = 1)
    
    tab["NAV"] = all_vl
    nav = tab["NAV"] #pd.DataFrame(tab["NAV"])
    planch = tab["Plancher"]
    coussin = tab["Coussin"]
    
    tabulate = pd.DataFrame([nav, planch, coussin]).T
        
    
    return tabulate

In [ ]:
a = monetarize(eonia, 100, 2000, 5)
a.plot()

## CPPI lorsque le poid attribué à l'actif risqué est constant et égal à 90%

In [ ]:
eonia_2000 = monetarize(eonia, 100,2000,5)
sxxr_2000 = normalize(sxxr,2000,5)

sxxr_1 = particularyear(sxxr_2000,2001)
eonia_1 = particularyear(eonia_2000,2001)
df_1 = concat(sxxr_1, eonia_1)

VL_1 = VL(df_1,0.9)
protect = get_list_protected(VL_1,2001,0.9,0)
protect
protect.iloc[:,0].plot(c="b")
protect.iloc[:,1].plot(c="r")

In [ ]:
protect

In [ ]:
w = 0.9
multiplier = 1/w

def protection(df:pd.DataFrame, m:multiplier = multiplier):
    tab = df
    # Initialisation des autres colonnes :
    tab["m"] = multiplier
    tab["Risk"] = 0
    tab["No Risk"] = 0
    n = len(tab)
    rang_couss = range(1,n+1)
    tab["Rang Coussin"] = rang_couss

    # Recherche de la première ligne à laquelle le coussin est négative ou nulle
    tabtravail = tab[["Coussin", "Rang Coussin"]]
    liste = []
    date = []
    compteur = 0
    for i in tabtravail.iloc[:,0]:
        compteur += 1
        if i <=0:
            liste.append(compteur)
            dateJ = tabtravail.iloc[compteur-1:compteur].index[0]
            date.append(dateJ)
    date_critique = date[0] # Au cas où l'on souhaite connaitre la date
    coussnegatif = min(liste)-1  # Le rang auquel le premier coussin soit négatif ou nul, -1 car ca commence à 0
    
    if len(date) > 0: # S'il existe une date à laquelle le coussin est négatif ou nul    
        # Mise à jour des colonnes Risk et No Risk

        risk = [x for x in tab["SXXR Index"]]
        norisk = [x for x in tab["EONIA Index"]]

        tab["Risk"] = risk
        tab["No Risk"] = norisk

        allocation = tab[["Risk", "No Risk", "EONIA"]]

        allocation = allocation.iloc[coussnegatif:,]
        allocation["No Risk"] = (1/m)*allocation["Risk"] + (1-1/m)*allocation["No Risk"]
        allocation["Risk"] = 0
    
        # Investir totalement sur l'actif non risqué
        rate = [x for x in allocation["EONIA"]]
        actnorisk = [x for x in allocation["No Risk"]]

        for i in range(len(allocation)):
            if i == 0:
                actnorisk[i] = actnorisk[i]
            else:
                actnorisk[i] = actnorisk[i-1] * (1 + rate[i-1]/(365*100))

        allocation["Actualized no risk"] = actnorisk 
        
        # concat
        
        vl = tab["VL"]
        vl = vl.iloc[:coussnegatif,]
        before = [round(x,3) for x in vl]
        
        after = [round(x,3) for x in allocation["Actualized no risk"]]
        
        all_vl = before + after
        
    #tab = tab.drop(["VL", "SXXR Index", "EONIA Index", "EONIA", "m", "Risk", "No Risk", "Rang Coussin"], axis = 1)
    
    tab["NAV"] = all_vl
    nav = tab["NAV"] #pd.DataFrame(tab["NAV"])
    planch = tab["Plancher"]
    
    tabulate = pd.DataFrame([nav, planch]).T

    
    return tabulate

In [ ]:
# Préparation de données

actif = df_1.drop("VL", axis = 1)
donnee = concat(protect, actif)
eonia_2001 = particularyear(eonia, 2001)
eonia_2001 = eonia_2001.rename(columns={"EONIA Index":"EONIA"})
donnee = concat(donnee, eonia_2001)
donnee

In [ ]:
riskdata = protection(donnee, multiplier)
riskdata

In [ ]:
riskdata["NAV"].plot(c="b", legend = "VL")
riskdata["Plancher"].plot(c="r", legend = "Plancher")
plt.legend()

In [ ]:
def get_list_protected_nav(df:pd.DataFrame, year:int, plancher:float=0.9, horizon:int=0):
    tabn = df[str(year-1):str(year-1)]
    tabn = pd.DataFrame(tabn.iloc[-1]).T
    tabz = df[str(year):str(year+horizon)]
    tab = pd.concat([tabn, tabz])
    all_plancher = []
    elemlist = []
    for elem in tab.iloc[:,0]:
        elemlist.append(elem)
        if elem == np.max(elemlist):
            planch = plancher * elem
            all_plancher.append(planch)
        else:
            planch = all_plancher[-1]
            all_plancher.append(planch)
            
    #result
    tab["Plancher"] = all_plancher
    tab["Coussin"] = tab["NAV"] - tab["Plancher"]
        
    return tab

In [ ]:
riskdata = pd.DataFrame(riskdata["NAV"])
#a = get_list_protected_nav(riskdata,2001,0.9,0)
riskdata

In [ ]:
class CPPI:
    def __init__(self, df1:pd.DataFrame = sxxr, df2:pd.DataFrame = eonia):
        self.df1 = df1
        self.df2 = df2
    
    def unifiedata(self, nav:float, year:int, horizon:int):
        eonia = monetarize(self.df2, nav,year,horizon)
        sxxr = normalize(self.df1,year,horizon)
        df = concat(sxxr, eonia)
        return df
    
    def protection(self, nav:float, year:int=2000, horizon:int=0, m:float=0.9):
        df_0 = self.unifiedata(nav,year,0)
        VL_0 = VL(df_0,0.9)
        protect_0 = get_list_protected(VL_0,year,m,0)
        
        return protect_0
    
    def unifiedprotection(self, nav:float,year:int=2000,horizon:int=0,m:float=0.9):
        # Year Y
        df_0 = self.protection(nav,year,0,m)
        df_0 = df_0.drop(df_0.index[-1])
        # Year Y+1
        df_1 = self.protection(nav,year+1,0,m)
        df_1 = df_1.drop(df_1.index[-1])
        # Year Y+2
        df_2 = self.protection(nav,year+2,0,m)
        df_2 = df_2.drop(df_2.index[-1])
        # Year Y+3
        df_3 = self.protection(nav,year+3,0,m)
        df_3 = df_3.drop(df_3.index[-1])
        # Year Y+4 (Final)
        df_4 = self.protection(nav,year+4,0,m)
        
        # Concat :
        df = pd.concat([df_0,df_1,df_2,df_3,df_4],axis=0)
        return df

In [ ]:
cppi = CPPI(sxxr, eonia)
data = cppi.unifiedata(100,2000,5)
data

In [ ]:
protect = cppi.unifiedprotection(100,2000,0,0.9)
if (min(protect.iloc[:,2]) <= 0):
    print("Faites attention, les coussins peuvent être négatifs ou nuls")
else:
    print("Les coussins sont positifs")


protect

In [ ]:
protect.iloc[:,0].plot(c="b", label="Valeur Liquidative")
protect.iloc[:,1].plot(c="r", label="Plancher")
protect.iloc[:,2].plot(c="g", label="Coussin")
plt.legend()

**Remarque** :
- Sur cette période de 5 ans (de 2000 à fin 2004), la VL peut être plus faible que le plancher, qui s'actualise chaque année.
- De ce fait, le coussin peut être négatif

## Protection

In [ ]:
def get_list_protected_nav(df:pd.DataFrame, year:int, plancher:float=0.9, horizon:int=0):
    tabn = df[str(year-1):str(year-1)]
    tabn = pd.DataFrame(tabn.iloc[-1]).T
    tabz = df[str(year):str(year+horizon)]
    tab = pd.concat([tabn, tabz])
    all_plancher = []
    elemlist = []
    for elem in tab.iloc[:,0]:
        elemlist.append(elem)
        if elem == np.max(elemlist):
            planch = plancher * elem
            all_plancher.append(planch)
        else:
            planch = all_plancher[-1]
            all_plancher.append(planch)
            
    #result
    tab["Plancher"] = all_plancher
    tab["Coussin"] = tab["NAV"] - tab["Plancher"]
        
    return tab

In [ ]:
class CPPIprotect:
    def __init__(self, df1:pd.DataFrame = sxxr, df2:pd.DataFrame = eonia):
        self.df1 = df1
        self.df2 = df2
    
    def unifiedata(self, nav:float=100, year:int=2000, horizon:int=5):
        eonia = monetarize(self.df2, nav,year,horizon)
        sxxr = normalize(self.df1,year,horizon)
        df = concat(sxxr, eonia)

        return df
    
    def get_data(self, nav:float=100, year:int=2000, horizon:int=5, m:float=0.9, h:int = 0):
        df = self.unifiedata(nav, year, horizon = 5)
        sxxr = pd.DataFrame(df["SXXR Index"])
        eonia = pd.DataFrame(df["EONIA Index"])
        
        sxxr = particularyear(sxxr, year+h)
        eonia = particularyear(eonia, year+h)
        
        data = concat(sxxr, eonia)
        val = VL(data, m)
        
        protect = get_list_protected(val, year+h, m, horizon = 5)
        actif = data.drop("VL", axis = 1)
        donnee = concat(protect, actif)
        
        
        eonia = particularyear(self.df2,year+h)
        eonia = eonia.rename(columns = {"EONIA Index":"EONIA"})
        donnee = concat(donnee, eonia)

        return donnee
    
    def test(self, nav:float=100, year:int=2000, horizon:int=5, m:float=0.9, h:int = 0):
        multiplier = 1/m
        
        # Year Y
        
        df0 = self.get_data(nav, year, horizon, m, h)
        riskdata0 = protection(df0, multiplier)
        
        # Year Y+1
        
        df1 = self.get_data(nav, year, horizon, m, h+1)
        riskdata1 = protection(df1,multiplier)
        
        # Year Y+2
        
        df2 = self.get_data(nav, year, horizon, m, h+2)
        riskdata2 = protection(df2,multiplier)
        
        # Year Y+3
        df3 = self.get_data(nav, year, horizon, m, h+3)
        riskdata3 = protection(df3,multiplier)
        
        # Year Y+4
        df4 = self.get_data(nav, year, horizon, m, h+4)
        riskdata4 = protection(df4,multiplier)
        
        df = pd.concat([riskdata0, riskdata1, riskdata2, riskdata3, riskdata4], axis = 0)

        return df
    
    def testtest(self, nav:float=100, year:int=2000, horizon:int=5, m:float=0.9, h:int = 0):
        multiplier = 1/m
        df = self.get_data(nav, year, horizon, m, h)
        riskdata = protection(df, multiplier)
        riskdata = pd.DataFrame(riskdata["NAV"])
        return riskdata
    
    def protec(self, nav:float=100, year:int=2000, horizon:int=5, m:float=0.9, h:int = 0):
        df = self.testtest(nav,year,horizon,m,h)
        data = get_list_protected_nav(df,year+h,m,0)
        return data
    
    def tess(self, nav:float=100, year:int=2000, horizon:int=5, m:float=0.9, h:int = 0):
        df0 = self.protec(nav, year, horizon, m, h)
        df1 = self.protec(nav, year, horizon, m, h+1)
        df2 = self.protec(nav, year, horizon, m, h+2)
        df3 = self.protec(nav, year, horizon, m, h+3)
        df4 = self.protec(nav, year, horizon, m, h+4)
        
        df = pd.concat([df0,df1,df2,df3,df4], axis = 0)
        return df

In [ ]:
a = CPPIprotect()
b = a.get_data(100,2005,0,0.9,1)
c = a.test(100,2005,0,0.9,1)
d = a.testtest(100,2000,0,0.9,1)
e = a.protec(100,2004,5,0.9,0)
e.plot()
#f = a.tess(100,2000,5,0.9,0)
#f.plot()
#c.plot()
#c = a.test()
#c.loc["2001", "NAV"].plot(c = "b", label = "NAV")
#c.loc["2001", "Plancher"].plot(c = "r", label = "Plancher")
#plt.legend()

In [ ]:
eonia_2000 = monetarize(eonia, 100,2000,5)
sxxr_2000 = normalize(sxxr,2000,5)

sxxr_1 = particularyear(sxxr_2000,2001)
eonia_1 = particularyear(eonia_2000,2001)
df_1 = concat(sxxr_1, eonia_1)

VL_1 = VL(df_1,0.9)
protect = get_list_protected(VL_1,2001,0.9,0)
protect
protect.iloc[:,0].plot(c="b")
protect.iloc[:,1].plot(c="r")

# Préparation de données

actif = df_1.drop("VL", axis = 1)
donnee = concat(protect, actif)
eonia_2001 = particularyear(eonia, 2001)
eonia_2001 = eonia_2001.rename(columns={"EONIA Index":"EONIA"})
donnee = concat(donnee, eonia_2001)
donnee

#riskdata = protection(donnee, multiplier)
#riskdata.plot()